# Entrainement d'un Classifieur en accord avec la Simulation Unity


### Importation des Bibliothèques

In [1]:
import pandas as pd
import numpy as np
from numpy import unique
from numpy import random
import seaborn as sns
from sklearn.preprocessing import StandardScaler 
import matplotlib.pyplot as plt 
%matplotlib inline

import time
import zmq
import array
from utils import myfct,mydisplay
import pickle

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import DBSCAN
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

### Programme Principale 
Cette partie du programme doit être lancé après la simulation Unity et avant la partie Segmentation en C++, le tout pour ne pas créer de problèmes en fonction des clients et des serveurs ZMQ

In [2]:
%%time


saving_for_experience = False
if (saving_for_experience):
    # Dossier dans le dossier Output ou nous allons sauvegarder nos données 
    #pour les réutiliser plus tards pour le plan d'experience
    saving_file = "env3_table3"   
    # Permet de ne pas écraser les fichiers préexistants si nous voulons
    # créer de nouvelles experiences pour un Batch
    starting_batch = 0


"""
Selection du model à utiliser, par défaut Random Forest est le plus efficace quand le taux d'erreur due à
l'effecteur du robot est inférieur à 20%. Cependant, grâce à l'algorithme d'oublie on est rarement au dessus de 
20% d'erreur dans notre base de donnée
"""
#model = SVC(kernel = 'rbf',probability=True,class_weight = 'balanced')
model = RandomForestClassifier()
#model = GaussianProcessRegressor(kernel = None)

"""
Hyperparamètres du batch d'experience:

nbBatch: 
Nombre d'"experience que l'on veux mener, en génerale, à par lors du plan d'experience, nbBatch = 1

nbScene: 
Nombre de fois que l'on va réorganiser les objets durant l'experience

nbIterPerScene: 
Nombre d'itération que l'on voudra effectuer entre 2 réorganisation des objets dans l'environnement
On obtient ainsi nbScene*nbIterPerScene interraction par experience
                
nb_model_for_removing_worst_SVP : 
Nombre de modèle utilisés pour l'algorithme d'oublie des outliers ttention car si le modèle demande 
beaucoup de calcule il est sans doute plus ertinent de baisser cette valeur
                            
proportion_of_completion: 
0.8 implique qu'au bout de 80% on fera le tri dans la base de données en utilisant l'algo 
d'oublie des Outliers. Dans tout les cas on refait le trie à la fin de l'eperience
                          
threshold_of_object: 
Taux d'objet dans la BDD au dela duquel on ne prends plus les SVP d'objets afin de ne pas 
déséquilibrer la base de données
"""
nbBatch = 1
nbScene = 15
nbIterPerScene = 5
nb_model_for_removing_worst_SVP = 1000
proportion_of_completion = 0.8
threshold_of_object = 0.42# 



batch_y_train_acc_exp = []
batch_y_train_acc_true = []
batch_X_train_acc_rough = []
selected_FPFH_number = 0

"""
Mise en place des serveur ZMQs pour la communication avec la partie segmentation en C++
Envoi de nbBatch, nbScene & nbIterPerScene pour l'initialisation
"""
context = zmq.Context()
socket = context.socket(zmq.REP)
socket.bind("tcp://*:5562")
message = socket.recv()
send_string = str(nbBatch)
socket.send_string(send_string)
message = socket.recv()
send_string = str(nbScene)
socket.send_string(send_string)
message = socket.recv()
send_string = str(nbIterPerScene)
socket.send_string(send_string)
print("data sent to C++")


"""
Début de l'experience
"""
for l in range (nbBatch):
    X_train_acc_rough = []
    y_train_acc_true = []
    y_train_acc_exp = []
    print("batch Numero ",l+1)
    for i in range(nbScene):
        for j in range(nbIterPerScene+1):
            
            if(j!=0):
                old_y_train_true = np.copy(y_train_true)
                old_X_train_rough = np.copy(X_train_rough)
                old_selected_FPFH_number = selected_FPFH_number
                
            #On recupère les FPFH calculé par la partie Segmentation sur le Nuage de points récupéré sous Unity
            message = socket.recv()
            message1 = array.array('B',message)
            nb_svp_moved,FPFHL_preshape = message1[0],message1[1:]
            

            if(nb_svp_moved!=0 and nb_svp_moved!=255):# si des SVP ont été déplacés et matchés:
                label = 1
                svp_moved_rough, FPFHL_preshape = message1[1:nb_svp_moved*2+1],message1[nb_svp_moved*2+1:]
                svp_moved = []
                for k in range(nb_svp_moved):
                    #On recupère les Numéro des svp ayant bougés; il sont codés à l'aide de 2 char
                    num_svp = int(svp_moved_rough[2*k]*255 + svp_moved_rough[2*k+1])
                    if(num_svp<=int(len(FPFHL_preshape)/49)):
                        #if(num_svp<=len(old_X_train_rough)):
                        svp_moved.append(int(svp_moved_rough[2*k]*255 + svp_moved_rough[2*k+1]))
                    print("svp_moved[k] :",svp_moved[k])
                    print("int(old_y_train_true[svp_moved[k]]) :",int(old_y_train_true[int(svp_moved[k])-1]))
            else:
                label = message1[0]#C'est le numéro du SVP d'arrière plan s'il n'y a pas eu de dépacement de SVP

            FPFH_LAB_L = np.reshape(FPFHL_preshape,(int(len(FPFHL_preshape)/49),49)).astype(int)
            X_train_rough = np.zeros((len(FPFH_LAB_L),48))
            y_train_true = np.zeros(len(FPFH_LAB_L))
            for k in range(len(FPFH_LAB_L)):
                X_train_rough[k][:48] = FPFH_LAB_L[k][:48]
                y_train_true[k] = FPFH_LAB_L[k][-1]
                 
            #On veux au moins trois FPFH d'objet avant d'utiliser la fonction d'acqusition
            #pour partir sur un modele relativement stable           
            if(sum(y_train_acc_exp)<3 or (len(y_train_acc_exp) - sum(y_train_acc_exp))<3 or len(y_train_acc_exp)<3):
                if(j!=0):
                    if(nb_svp_moved == 0):
                        y_train_acc_exp.append(0)
                        y_train_acc_true.append(int(old_y_train_true[old_selected_FPFH_number-1]))
                        X_train_acc_rough.append(old_X_train_rough[old_selected_FPFH_number-1])
                        
                    else:
                        for k in range(int(nb_svp_moved)):
                            num_sv = int(svp_moved[k])-1
                            y_train_acc_exp.append(1)
                            y_train_acc_true.append(int(old_y_train_true[num_sv]))
                            X_train_acc_rough.append(old_X_train_rough[num_sv])

               #On commence donc par selectionner nos SVP selon une loi uniforme
                selected_FPFH_number = int(random.uniform(0,len(X_train_rough)))
            
            #On entre dans ce else une fois qu"on a sufisement de données pour que le classifieur puisse utiliser la 
            #fonction d'acquisition pour la selection des futurs SVP sur lesquels activer la primitive de poussée
            else:
                if(j!=0):
                    if(nb_svp_moved == 0):
                        y_train_acc_exp.append(0)
                        y_train_acc_true.append(int(old_y_train_true[old_selected_FPFH_number-1]))
                        X_train_acc_rough.append(old_X_train_rough[old_selected_FPFH_number-1])

                        
                    else:
                        if(sum(y_train_acc_exp)/len(y_train_acc_exp)<threshold_of_object):
                            for k in range(int(nb_svp_moved)):
                                num_sv = int(svp_moved[k])-1
                                y_train_acc_exp.append(1)
                                y_train_acc_true.append(int(old_y_train_true[num_sv]))
                                X_train_acc_rough.append(old_X_train_rough[num_sv])

                                

            
                #On réentraine le model avec toutes les données accumulées jusqu'a présent
                X_train_acc = myfct.normalize(X_train_acc_rough)
                model.fit(X_train_acc,y_train_acc_exp)  
                X_train = myfct.normalize(X_train_rough)
                #On prédit les probabilités d'appartenir à un objet ou au background de tout les SVP présent dans 
                #la scène car on en aura besoin pour la fonction d'acquisition
                preds_proba = model.predict_proba(X_train)
                #Fonction d'acquisition
                selected_FPFH_number = myfct.getRelevantFPFH(preds_proba,y_train_acc_exp,threshold_of_object)

            #Si on arrive à ce niveau de l'experience (ici 80%) on lance l'algo doublie des SVP non cohérents
            if(i*j==int(proportion_of_completion*nbScene*nbIterPerScene) and len(y_train_acc_exp)>20):
                X_train_acc_rough,y_train_acc_exp,y_train_acc_true = myfct.forget_worst_sample(
                    model,nb_model_for_removing_worst_SVP,X_train_acc_rough,y_train_acc_exp,y_train_acc_true)
                
            #Envoie du SVP selectionné par la fonction d'acquisition à la partie Segmentation 
            socket.send_string(str(selected_FPFH_number))
            if(label!=255):        
                print(" y_train_true",old_y_train_true[old_selected_FPFH_number])
                
    #on relance l'algo doublie des SVP non cohérents à la fin de l'experience
    if(len(y_train_acc_exp)>20):
        X_train_acc_rough,y_train_acc_exp,y_train_acc_true = myfct.forget_worst_sample(
                    model,nb_model_for_removing_worst_SVP,X_train_acc_rough,y_train_acc_exp,y_train_acc_true) 
    
    #On récupère les échantillons, leur label déduit grace à l'effecteur ainsi que leur véritable label pour comparer
    batch_y_train_acc_exp.append(y_train_acc_exp)
    batch_y_train_acc_true.append(y_train_acc_true)
    batch_X_train_acc_rough.append(X_train_acc_rough)
    

    
    
    
    if(saving_for_experience):#Sauvegarde des données pour le plan d'experience

        path_y_train_acc_exp = "../Plan_exp/" + saving_file + "/batch_num_"+str(l+starting_batch)+"y_train_acc_exp"
        myfct.writePickler(path_y_train_acc_exp,y_train_acc_exp)
        path_y_train_acc_true = "../Plan_exp/" + saving_file + "/batch_num_"+str(l+starting_batch)+"y_train_acc_true"
        myfct.writePickler(path_y_train_acc_true,y_train_acc_true)
        path_X_train_acc_rough = "../Plan_exp/" + saving_file + "/batch_num_"+str(l+starting_batch)+"X_train_acc_rough"
        myfct.writePickler(path_X_train_acc_rough,X_train_acc_rough)


data sent to C++
batch Numero  1
 y_train_true 0.0
 y_train_true 0.0
 y_train_true 0.0
 y_train_true 0.0
svp_moved[k] : 501
int(old_y_train_true[svp_moved[k]]) : 1
 y_train_true 0.0
 y_train_true 0.0
svp_moved[k] : 335
int(old_y_train_true[svp_moved[k]]) : 1
 y_train_true 0.0
svp_moved[k] : 580
int(old_y_train_true[svp_moved[k]]) : 0
svp_moved[k] : 450
int(old_y_train_true[svp_moved[k]]) : 0
svp_moved[k] : 591
int(old_y_train_true[svp_moved[k]]) : 1
svp_moved[k] : 321
int(old_y_train_true[svp_moved[k]]) : 0
svp_moved[k] : 326
int(old_y_train_true[svp_moved[k]]) : 0
svp_moved[k] : 452
int(old_y_train_true[svp_moved[k]]) : 1
 y_train_true 0.0
svp_moved[k] : 619
int(old_y_train_true[svp_moved[k]]) : 1
svp_moved[k] : 119
int(old_y_train_true[svp_moved[k]]) : 1
svp_moved[k] : 171
int(old_y_train_true[svp_moved[k]]) : 0
svp_moved[k] : 175
int(old_y_train_true[svp_moved[k]]) : 1
svp_moved[k] : 125
int(old_y_train_true[svp_moved[k]]) : 0
svp_moved[k] : 137
int(old_y_train_true[svp_moved[k]]) :

KeyboardInterrupt: 

### Observation rapide du taux d'erreur dans nos bases de données 
Nous en avons plusieurs si nous avons positionnés nbBatch >1

In [3]:
for i in range(len(batch_y_train_acc_exp)):
    print("Batch Numero :",i)
    print("nb de données apres oublie :",len(batch_y_train_acc_exp[i]))
    print("Taux d'erreur apres oublie :",myfct.error_pred(batch_y_train_acc_exp[i][:],batch_y_train_acc_true[i][:]))

### Quelques Courbes rapide pour observer les résultats obtenus
Attention cependant car la pluspart de ces courbes nécessite d'utiliser nbBatch > 1 afin de mettre l'écart type en évidence

In [4]:
FPFHL_folder = "env3_table3"
nb_scenes = 13#Nombre de scène pour l'entrainement
#Attention de prendre le même environnement pour l'entrainement et pour les tests

###DISPLAY Precision, Recall & Accuracy of the robotic arm
#title = "Interpretation des résultats de la simulation"
#mydisplay.displayNormalizedResults(title,batch_y_train_acc_exp,batch_y_train_acc_true,batch_X_train_acc_rough)

###DISPLAY Precision, Recall & Accuracy from a database without forgetting
#title = "Phase d'exploitation à partir des données rééquilibrées de 20 scènes sans oublie d'outlier"

###Une fois qu'on à notre base de données, on peut s'ammuser à changer le classifieur pour observer les résultats
#from sklearn.neighbors import KNeighborsClassifier
#model = KNeighborsClassifier(n_neighbors=10)
#model = SVC(kernel = 'rbf',probability=True,class_weight = 'balanced')
#mydisplay.displayNormalizedResults2(model,FPFHL_folder,nb_scenes,title,noForget_batch_y_train_acc_exp,noForget_batch_y_train_acc_true,noForget_batch_X_train_acc_rough)


###DISPLAY Precision, Recall & Accuracy from a database
#title = "Phase d'exploitation à partir des données rééquilibrées de 20 scènes avec oublie"
title = "Phase d'exploitation"

#from sklearn.neighbors import KNeighborsClassifier
#model = KNeighborsClassifier(n_neighbors=10)
#model = SVC(kernel = 'rbf',probability=True,class_weight = 'balanced')
model = RandomForestClassifier()
mydisplay.displayNormalizedResults2(model,FPFHL_folder,nb_scenes,title,batch_y_train_acc_exp,batch_y_train_acc_true,batch_X_train_acc_rough)


###DISPLAY Precision, Recall & Accuracy from truth
#title = "Phase d'exploitation à partir des données rééquilibrées de 20 scènes avec oublie et veritable label"

#from sklearn.neighbors import KNeighborsClassifier
#model = KNeighborsClassifier(n_neighbors=10)
#model = SVC(kernel = 'rbf',probability=True,class_weight = 'balanced')
#model = RandomForestClassifier()
#mydisplay.displayNormalizedResults2(model,FPFHL_folder,nb_scenes,title,batch_y_train_acc_true,batch_y_train_acc_true,batch_X_train_acc_rough)



###DISPLAY Error Rate
title = "Error Rates au cours de l'apprentissage testé sur une base de données rééquilibrées de 20 scènes\n"
y_lim = (0,0.3)
num_batch = 0
mydisplay.error_rates_on_Dataset(num_batch,model,y_lim,FPFHL_folder,nb_scenes,title,batch_y_train_acc_exp,batch_y_train_acc_true,batch_X_train_acc_rough)

IndexError: list index out of range

### Test de segmentation sur une scène sauvegardé par la partie Segmentation dans le dossier outputs

In [ ]:
nomDossier = "env3_table3"
numIter = "3"
numeroDeScene = "6"
paths_to_output = "../outputs/"

Ici on récupère les données de la scène puis on entraine un classifieur avec la base de donnée que nous avons obtenus précedement lors de l'experience N°numBatch

In [ ]:
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import classification_report,confusion_matrix

#On entraine un modèle grace aux données recuperé lors de l'experience N°numBatch
numBatch = 0
#batch_y_train_acc_true[numBatch][0] = 1#Souvent faux mais cela permet d'afficher les courbes dès les 1eres interactions 
model.fit(myfct.normalize(batch_X_train_acc_rough[numBatch][:]),batch_y_train_acc_exp[numBatch][:])

sceneTest=pd.read_csv("../outputs/"+nomDossier+"/fpfh/fpfh_scene"+str(numeroDeScene)+"iter"+str(numIter)+".txt")
sceneTest.head()



#NORMALISE LES DONN2ES TESTS!!!!
scaler=StandardScaler() 
scaler.fit(sceneTest.drop('label',axis=1))
scaled_features = scaler.transform(sceneTest.drop('label',axis=1))
sceneTest_feat = pd.DataFrame(scaled_features,columns=sceneTest.columns[:-1])
sceneTest_feat.head()


testPrediction = model.predict(myfct.normalize(scaled_features)) #Ici on choisit le model que l'on veut tester
testProba = model.predict_proba(myfct.normalize(scaled_features))
#print(confusion_matrix(y_test,pred))
testReality = np.array(sceneTest["label"])
testReality = np.concatenate(([0],testReality))

### Affichage du nuage de point selectionné

In [ ]:


#segmented = mydisplay.read_xyzrgbls(paths_to_output+str(nomDossier)+"/vccs/vccs_scene"+str(numeroDeScene)+"iter"+numIter+".xyzrgbls")
segmented = mydisplay.read_xyzrgbls(paths_to_output+str(nomDossier)+"/nuageDisplay/nuage_scene"+str(numeroDeScene)+"iter"+numIter+".xyzrgbls")
svp_centroides = mydisplay.read_xyzrgbls(paths_to_output+str(nomDossier)+"/centroides/centroides_scene"+str(numeroDeScene)+"iter"+numIter+".xyzrgbls")
sceneTest=pd.read_csv("../outputs/"+nomDossier+"/fpfh/fpfh_scene"+str(numeroDeScene)+"iter"+str(numIter)+".txt")
X1,Y1,Z1,X2,Y2,Z2 = [],[],[],[],[],[]
for i in range(len(segmented)):
    X1.append(segmented[i][0])
    Y1.append(segmented[i][1])
    Z1.append(0)#segmented[i][2])
    
    
        
colors = []
for i in range (len(segmented)):
    colors.append((segmented[i][3]/255, segmented[i][4]/255, segmented[i][5]/255))



colors = np.array(colors)

categories = []
for i in range(len(segmented)):
    categories.append(i)
    
fig = plt.figure(figsize = (18,15))
ax = fig.gca(projection='3d')
ax.view_init(90, 270)
ax.scatter(X1, Y1, Z1, c=colors[categories], marker='.')

### Affichage des prévisions du classifieur
Les points jaunes représentent les Centroïdes des SVP percu comme appartenant à un objet selon notre classifieur

In [ ]:
#segmented = mydisplay.read_xyzrgbls(paths_to_output+str(nomDossier)+"/vccs/vccs_scene"+str(numeroDeScene)+"iter"+numIter+".xyzrgbls")
segmented = mydisplay.read_xyzrgbls(paths_to_output+str(nomDossier)+"/nuageDisplay/nuage_scene"+str(numeroDeScene)+"iter"+numIter+".xyzrgbls")
svp_centroides = mydisplay.read_xyzrgbls(paths_to_output+str(nomDossier)+"/centroides/centroides_scene"+str(numeroDeScene)+"iter"+numIter+".xyzrgbls")
X1,Y1,Z1,X2,Y2,Z2 = [],[],[],[],[],[]

for i in range(len(segmented)):
    X1.append(segmented[i][0])
    Y1.append(segmented[i][1])
    Z1.append(0)#segmented[i][2])
    
for i in range(len(testPrediction)):
    if(testPrediction[i]):
        X2.append(svp_centroides[i][0])
        Y2.append(svp_centroides[i][1])
        Z2.append(0)#svp_centroides[i][2]+1000)
    

        
colors = []
for i in range (len(segmented)):
    colors.append((segmented[i][3]/255, segmented[i][4]/255, segmented[i][5]/255))



colors = np.array(colors)

categories = []
for i in range(len(segmented)):
    categories.append(i)
    
fig = plt.figure(figsize = (18,15))
ax = fig.gca(projection='3d')
ax.view_init(90, 270)
ax.scatter(X1, Y1, Z1, c=colors[categories], marker='.')
ax.scatter(X2, Y2, Z2, c='yellow',linewidths = 4, marker='.')

### Affichage des prévisions du classifieur avec seuil
Les points bleus représentent les Centroïdes des SVP percu comme appartenant à un objet selon notre classifieur avec une prédiction suppérieur au seuil(threshold) selectionné

In [ ]:
threshold = 0.97


#segmented = mydisplay.read_xyzrgbls(paths_to_output+str(nomDossier)+"/vccs/vccs_scene"+str(numeroDeScene)+"iter"+numIter+".xyzrgbls")
segmented = mydisplay.read_xyzrgbls(paths_to_output+str(nomDossier)+"/nuageDisplay/nuage_scene"+str(numeroDeScene)+"iter"+numIter+".xyzrgbls")
svp_centroides = mydisplay.read_xyzrgbls(paths_to_output+str(nomDossier)+"/centroides/centroides_scene"+str(numeroDeScene)+"iter"+numIter+".xyzrgbls")
X1,Y1,Z1,X2,Y2,Z2 = [],[],[],[],[],[]

for i in range(len(segmented)):
    X1.append(segmented[i][0])
    Y1.append(segmented[i][1])
    Z1.append(0)#segmented[i][2])
    
for i in range(len(testProba)):
    if(testProba[i][1]>threshold):
        X2.append(svp_centroides[i][0])
        Y2.append(svp_centroides[i][1])
        Z2.append(0)#svp_centroides[i][2]+1000)
        

        
colors = []
for i in range (len(segmented)):
    colors.append((segmented[i][3]/255, segmented[i][4]/255, segmented[i][5]/255))



colors = np.array(colors)

categories = []
for i in range(len(segmented)):
    categories.append(i)
    
fig = plt.figure(figsize = (18,15))
ax = fig.gca(projection='3d')
ax.view_init(90, 270)
ax.scatter(X1, Y1, Z1, c=colors[categories], marker='.')
ax.scatter(X2, Y2, Z2, c='blue',linewidths = 4, marker='.')

### Affichage du nuage de point selectionné dans le referentiel monde

In [ ]:


segmented = mydisplay.read_xyzrgbls(paths_to_output+str(nomDossier)+"/vccs/vccs_scene"+str(numeroDeScene)+"iter"+numIter+".xyzrgbls")

X,Y,Z = [],[],[]
for i in range(len(segmented)):
    X.append(segmented[i][0])
    Y.append(segmented[i][1])
    Z.append(segmented[i][2])
    
svp_obj = []
for i in range(len(testPrediction)):
    if(testPrediction[i]):
        svp_obj.append(i+1) 
        
colors = []
for i in range (len(segmented)):
    colors.append((segmented[i][3]/255, segmented[i][4]/255, segmented[i][5]/255))
    #for j in range(len(svp_obj)):
        #if(segmented[i][7] == svp_obj[j]):
        #    colors[-1] = (1.0, 1.0, 0.0)


colors = np.array(colors)

categories = []
for i in range(len(segmented)):
    categories.append(i)
    
fig = plt.figure(figsize = (18,15))
ax = fig.gca(projection='3d')
ax.view_init(90-35, 270)
ax.scatter(X, Y, Z, c=colors[categories], marker='.')
#ax.scatter(400, 400, 1000, c='yellow',linewidths = 5, marker='o')

### Affichage des prévisions du classifieur dans le referentiel monde
Les SVP percu comme appartenant à un objet selon notre classifieur sont coloriés en Jaune
Utilisation d'un seuil(threshold)

In [ ]:
threshold = 0.93


segmented = mydisplay.read_xyzrgbls(paths_to_output+str(nomDossier)+"/vccs/vccs_scene"+str(numeroDeScene)+"iter"+numIter+".xyzrgbls")

X,Y,Z = [],[],[]
for i in range(len(segmented)):
    X.append(segmented[i][0])
    Y.append(segmented[i][1])
    Z.append(segmented[i][2])
    
svp_obj = []
for i in range(len(testProba)):
    if(testProba[i][1]>threshold):
        svp_obj.append(i+1) 
        
colors = []
for i in range (len(segmented)):
    colors.append((segmented[i][3]/255, segmented[i][4]/255, segmented[i][5]/255))
    for j in range(len(svp_obj)):
        if(segmented[i][7] == svp_obj[j]):
            colors[-1] = (1.0, 1.0, 0.0)


colors = np.array(colors)

categories = []
for i in range(len(segmented)):
    categories.append(i)
    
fig = plt.figure(figsize = (18,15))
ax = fig.gca(projection='3d',proj_type = 'persp')

ax.view_init(90-35, -90)
#ax.zaxis_inverted
ax.scatter(X, Y, Z, c=colors[categories], marker='.')

### Affichage des Supervoxels persistants dans le referentiel Monde
Ici chaque supervoxel est associé à une couleur particulière

In [ ]:


segmented = mydisplay.read_xyzrgbls(paths_to_output+str(nomDossier)+"/vccs/vccs_scene"+str(numeroDeScene)+"iter"+numIter+".xyzrgbls")

X,Y,Z = [],[],[]
for i in range(len(segmented)):
    X.append(segmented[i][0])
    Y.append(segmented[i][1]+((-2.82/4.5)+1)*128)
    Z.append(segmented[i][2]+((+3.42/4.5)+1)*128)
    
colors = []
for i in range (3000):
    colors.append((random.uniform(0,1), random.uniform(0,1), random.uniform(0,1), 1.0))
colors = np.array(colors)

categories = []
for i in range(len(segmented)):
    categories.append(int(segmented[i][-1]))
    
fig = plt.figure(figsize = (18,15))
ax = fig.gca(projection='3d')
ax.view_init(90-35, -90)
ax.scatter(X, Y, Z, c=colors[categories], marker='.')

### Affichage des SVP appartenant à un objet dans le referentiel Monde
Les SVP appartenant réellement à un objet sont coloriés en Jaune

In [ ]:


sceneTest=pd.read_csv("../outputs/"+nomDossier+"/fpfh/fpfh_scene"+str(numeroDeScene)+"iter"+str(numIter)+".txt")

scaler=StandardScaler() 
scaler.fit(sceneTest.drop('label',axis=1))
scaled_features = scaler.transform(sceneTest.drop('label',axis=1))
sceneTest_feat = pd.DataFrame(scaled_features,columns=sceneTest.columns[:-1])


sv_obj = []
for i in range(len(sceneTest)):
    if(sceneTest["label"][i] == 1):
        sv_obj.append(i+1)


segmented = mydisplay.read_xyzrgbls(paths_to_output+str(nomDossier)+"/vccs/vccs_scene"+str(numeroDeScene)+"iter"+numIter+".xyzrgbls")

X,Y,Z = [],[],[]
for i in range(len(segmented)):
    X.append(segmented[i][0])
    Y.append(segmented[i][1])
    Z.append(segmented[i][2])
    
colors = []
for i in range (len(segmented)):
    colors.append((segmented[i][3]/256, segmented[i][4]/256, segmented[i][5]/256))
    for j in range(len(sv_obj)):
        if(segmented[i][7] == sv_obj[j]):
            colors[-1] = (1, 1, 0)


colors = np.array(colors)

categories = []
for i in range(len(segmented)):
    categories.append(i)
    
fig = plt.figure(figsize = (18,15))

ax = fig.gca(projection='3d')

ax.view_init(90-35, 270)
ax.scatter(X, Y, Z, c=colors[categories], marker='.')